## Problem:

- In out problem, need to detect the nearest pedestrian road to see if the car is violating or not 
- need to know which part is the pedestrian road and which part is the car road  

In [1]:
import os
import sys
import cv2
import time
import mmcv
import torch
import torch.nn as nn
import numpy as np
# import ffmpeg
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
from crfseg import CRF

repo_path = Path(".").absolute().parent

if os.system == "nt":
    data_path = Path("D:\Datas\parking_violation")
else:
    data_path = repo_path.parent / "data" / "parking_violation"
sys.path.append(str(repo_path))

package repo tree

```
├── data
│   ├── mmseg
│   │   └── checkpoints
│   └── parking_violation
├── mmsegmentation
│   └── configs
│       └── resnest
└── parking_violation
    ├── utils.py
    └── notebooks
        └── 04_Segmentation.ipynb
```

don't forget to download the weight first

**backbone: resnest**

```
!wget https://download.openmmlab.com/mmsegmentation/v0.5/resnest/deeplabv3plus_s101-d8_512x1024_80k_cityscapes/deeplabv3plus_s101-d8_512x1024_80k_cityscapes_20200807_144429-1239eb43.pth -P ~/code/data/mmseg/checkpoints/
```

**backbone: R-18-D8**

```
!wget https://download.openmmlab.com/mmsegmentation/v0.5/deeplabv3plus/deeplabv3plus_r18-d8_512x1024_80k_cityscapes/deeplabv3plus_r18-d8_512x1024_80k_cityscapes_20201226_080942-cff257fe.pth -P ~/code/data/mmseg/checkpoints/
```

**backbone: R-50-D8**

```
!wget https://download.openmmlab.com/mmsegmentation/v0.5/deeplabv3plus/deeplabv3plus_r50-d8_512x1024_40k_cityscapes/deeplabv3plus_r50-d8_512x1024_40k_cityscapes_20200605_094610-d222ffcd.pth -P ~/code/data/mmseg/checkpoints/
```

In [2]:
from mmseg.apis import inference_segmentor, init_segmentor, show_result_pyplot
from mmseg.datasets import CityscapesDataset

model_config = "deeplabv3plus"
backbone_config = "r50-d8"
backbone_dict = {
    "r50-d8": {
        "config_dir": "deeplabv3plus",
        "config": "deeplabv3plus_r50-d8_512x1024_40k_cityscapes.py",
        "checkpoint": "deeplabv3plus_r50-d8_512x1024_40k_cityscapes_20200605_094610-d222ffcd.pth"
    },
    "r18-d8": {
        "config_dir": "deeplabv3plus",
        "config": "deeplabv3plus_r18-d8_512x1024_80k_cityscapes.py",
        "checkpoint": "deeplabv3plus_r18-d8_512x1024_80k_cityscapes_20201226_080942-cff257fe.pth"
    },
    "resnest": {
        "config_dir": "resnest",
        "config": "deeplabv3plus_s101-d8_512x1024_80k_cityscapes.py",
        "checkpoint": "deeplabv3plus_s101-d8_512x1024_80k_cityscapes_20200807_144429-1239eb43.pth"
    } 
}


config_path = repo_path.parent / "mmsegmentation" / "configs" / backbone_dict[backbone_config]["config_dir"]
checkpoint_path = repo_path.parent / "data" / "mmseg" / "checkpoints"
if not checkpoint_path.exists():
    checkpoint_path.mkdir(parents=True)

config_file = str(config_path / backbone_dict[backbone_config]["config"])
checkpoint_file = str(checkpoint_path / backbone_dict[backbone_config]["checkpoint"])
crf_checkpoint_file = str(checkpoint_path / f"crf_{backbone_config}.pth")

In [7]:
# Preprocess video: Origin size is 720x1280
height, width = 480, 640
video_name = "sample2"
# height, width = 720, 1280
video_path = str(data_path / "origin" / f"{video_name}.mp4")
resized_video_path = str(data_path / f"{video_name}_{height}x{width}.mp4")
resized_frames_path = data_path / f"{video_name}_{height}x{width}"
if not resized_frames_path.exists():
    resized_frames_path.mkdir()

# uncomment underline to resize video
# mmcv.resize_video(video_path, resized_video_path, (width, height))

# uncomment underline to preprocessing
frames_path = resized_frames_path / "img_dir"
# if list(frames_path.glob("*.jpg")):
#     for file in frames_path.glob("*.jpg"):
#         file.unlink()
        
# video = mmcv.VideoReader(resized_video_path)

# ratio = 0.65
# bboxes = np.array([0, 0, width, int(height*ratio)])
# for i, (frame) in tqdm(enumerate(video, 1), total=len(video), desc="Extracting"):
#     frame = mmcv.imcrop(frame, bboxes)
#     mmcv.imwrite(frame, str(frames_path / f"{i:06d}.jpg"))

## Load Model

In [8]:
from mmseg.datasets import build_dataloader, CustomDataset

In [9]:
class SegModelCRF(nn.Module):
    def __init__(self, config_file, model_checkpoint_file, crf_checkpoint_file, device):
        super().__init__()
        self.d = device
        self.seg_model = init_segmentor(config_file, model_checkpoint_file, device=device)
        self.cfg = self.seg_model.cfg
        
        self.crf = CRF(n_spatial_dims=2, returns="log-proba").to(device)
        self.crf.load_state_dict(torch.load(crf_checkpoint_file))
        
    def forward(self, data):
        img = data["img"][0].data.to(self.d)
        img_meta = data["img_metas"][0].data[0]
        x = self.seg_model.inference(img, img_meta, rescale=True)
        x = self.crf(x, display_tqdm=False)
        x = x.argmax(1)
        return x.cpu().numpy()

In [10]:
model = SegModelCRF(config_file, checkpoint_file, crf_checkpoint_file, device="cuda")

In [11]:
batch_size = 4

img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)

pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(width, height),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='Normalize', **img_norm_cfg),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),            
        ]
    )
]

dataset = CustomDataset(
    pipeline=pipeline,
    img_dir="img_dir",
    data_root=str(resized_frames_path), 
    classes=CityscapesDataset.CLASSES, 
    palette=CityscapesDataset.PALETTE,
    test_mode=True)

data_loader = build_dataloader(
    dataset, samples_per_gpu=batch_size, 
    workers_per_gpu=1, dist=False, shuffle=False, dataloader_type="DataLoader")

2021-01-19 13:55:02,463 - mmseg - INFO - Loaded 3622 images


---

In [12]:
def post_process(x, pleft=0, ptop=30, pright=200, pbottom=30):
    h, w, _ = x.shape
    img = mmcv.impad(x, padding=(pleft, ptop, pright, pbottom), pad_val=0)
    # pts: lu, ld, rd, ru & each point = (x, y)
    pts = [np.array([(w,ptop),(w,ptop+h),(w+pright,ptop+h),(w+pright,ptop)])]
    cv2.fillPoly(img, pts=pts, color=(255,255,255))
    for i, (cls, clr) in enumerate(class2color.items()):
        percent = i/len(class2color)
        pos_x = int(w + 0.1*pright)
        pos_y = int(percent*h + ptop) + int(h*0.025)
        cv2.circle(img, (pos_x, pos_y), int(h*0.01), clr, thickness=-1)
        cv2.putText(img, cls, (pos_x+int(h*0.025), pos_y+3), cv2.FONT_HERSHEY_SIMPLEX, h*0.002, color=(0,0,0))

    return img

get_img_path = lambda x: [m.get("filename") for m in x["img_metas"][0].data[0]]

output_dir = resized_frames_path / "output" / backbone_config
if not output_dir.exists():
    output_dir.mkdir()
if list(output_dir.glob("*.jpg")):
    for file in output_dir.glob("*.jpg"):
        file.unlink()    

class2color = dict(zip(*[dataset.CLASSES, dataset.PALETTE]))
pleft, ptop, pright, pbottom = 0, 50, 150, 50
bgr_palette = np.array(dataset.PALETTE)[..., ::-1]  # need to convert to bgr

pbar = tqdm(desc="Processing:", total=len(data_loader)*batch_size)

for data in data_loader:
    result = model(data)
    torch.cuda.empty_cache()
    for img_p, res in zip(get_img_path(data), result):
        frame = mmcv.bgr2rgb(mmcv.imread(img_p))
        overlay_img = model.seg_model.show_result(frame, [res], palette=bgr_palette, show=False)
        img = post_process(overlay_img)
        
        # 이미지 이름에 맞춰서 저장 img_p
        mmcv.imwrite(img, str(output_dir / f"{video_name}-{Path(img_p).name}"))
    pbar.update(batch_size)

Processing::   0%|          | 0/3624 [00:00<?, ?it/s]/home/simonjisu/miniconda3/lib/python3.7/site-packages/crfseg/model.py:91: UserWarning: Mixed memory format inputs detected while calling the operator. The operator will output contiguous tensor even if some of the inputs are in channels_last format. (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629403081/work/aten/src/ATen/native/TensorIterator.cpp:918.)
  x = negative_unary - x
/home/simonjisu/miniconda3/lib/python3.7/site-packages/mmseg/models/segmentors/base.py:267: UserWarning: show==False and out_file is not specified, only result image will be returned
  warnings.warn('show==False and out_file is not specified, only '
Processing:: 100%|██████████| 3624/3624 [08:08<00:00,  8.67it/s]

In [ ]:
from moviepy.editor import concatenate_videoclips, ImageClip
imgs = sorted(list(map(str, output_dir.glob("*.jpg"))))

clips = [ImageClip(m).set_duration(1/30) for m in imgs]
video_output_path = str(data_path / f"sample1_{height}x{width}_{backbone_config}_result.mp4")

concat_clip = concatenate_videoclips(clips, method="compose")
concat_clip.write_videofile(video_output_path, fps=30) #int(video.fps/step)

In [58]:
import ipywidgets as wd
from IPython.display import display, HTML

In [60]:
HTML("""
<video width="640" height="300" controls>
  <source src="{}" type="video/mp4">
</video>
""".format(video_output_path))

---

**To be updated**

## Process Segmentation on Video

Problem: Image at each frame contains a lot of noise. Need some post-processing before get the final output.

### Conditional Random Field

#### CRF as CNN

- Paper: https://arxiv.org/abs/1210.5644
- Article Refernece: https://arxiv.org/abs/1502.03240

<img src="./imgs/04_CRF02.png" width=480>

A CRF, used in the context of pixel-wise label prediction, models pixel labels as random variables that form a Markov Random Field(MRF) when conditioned upon a global observation. The global observation is usually taken to be the image.

* Let $X_i$ be the random variable associated to pixel $i$, which represents the label assigned to the pixel $i$ and
can take any value from a pre-defined set of labels $L = \{l_1, l_2, \cdots , l_L\}$. 
* Let $X$ be the vector formed by the random variables $X_1, X_2, \cdots , X_N$ , where $N$ is the number of pixels in the image. 
* Given a graph $G = (V, E)$, where $V = \{X_1, X_2, \cdots , X_N \}$, and a global observation (image) $I$, the pair $(I, X)$ can be modelled as a CRF characterized by a Gibbs distribution of the form $P(X = x|I) = \dfrac{1}{Z(I)}\exp\big(−E(x|I)\big)$. 
* Here $E(x)$ is called the energy of the configuration $x \in \mathcal{L}^N$ and $Z(I)$ is the partition function.
* From now on, we drop the conditioning on $I$ in the notation for convenience.
* In the fully connected pairwise CRF model of [29](https://arxiv.org/abs/1210.5644), [pdf](http://swoh.web.engr.illinois.edu/courses/IE598/handout/fall2016_slide15.pdf), the energy of a label assignment x is given by: 
    $$ E(x) = \sum_{i}{\psi}_u (x_i) + \sum_{i < j} \psi_p (x_i, x_j)$$
    * where the unary energy components $\psi_u(x_i)$ measure the inverse likelihood (and therefore, the cost) of the pixel $i$ taking the label $x_i$ $\rightarrow$ label score
    * pairwise energy components $\psi_p(x_i, x_j)$ measure the cost of assigning labels $x_i, x_j$ to pixels $i, j$ simultaneously $\rightarrow$ data-dependent smoothing term that encourages assiging similar lavels to pixels with similar properties.
*  As was done in [29](https://arxiv.org/abs/1210.5644), we model pairwise potentials as weighted Gaussians:
    $$ \psi_p(x_i, x_j) = \mu(x_i, x_j) \sum_{m=1}^M w^{(m)}k_G^{(m)}(f_i, f_j)$$
    * where each $k^{(m)}_G$ for $m = 1, \cdots, M$ is a Gaussian kernel applied on feature vectors
    * The feature vector of pixel $i$, denoted by $f_i$, is derived from image features such as spatial location and RGB values [29].
    * The function $µ(., .)$, called the label compatibility function, captures the compatibility between different pairs of labels as the name implies.
* Minimizing the above CRF energy $E(x)$ yields the most probable label assignment $x$ for the given image. Since this exact minimization is intractable, a mean-field approximation to the CRF distribution is used for approximate maximum posterior marginal inference.
*  It consists in approximating the CRF distribution $P(X)$ by a simpler distribution $Q(X)$, which can be written as the product of independent marginal distributions, i.e., $Q(X) = \prod_{i} Q_i(X_i)$.

<img src="./imgs/04_CRF01.png" width=480>

* input=unary potentials: (B, K, *spatial)
* Normalizing: softmax(dim=1)
* Message Passing: apply Gaussian Kernel for each image (B, K, *spatial) `Conv1d` 
* Compatibility Transform: torch.einsum(input, compatibility) (B, K, *spatial)
    * compatibility: negative identity matrix (K, K)
* adding unary potentials

#### CRF as RNN

- Paper: https://arxiv.org/abs/1502.03240
- Code: https://github.com/sadeepj/crfasrnn_pytorch

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from tqdm import tqdm
from mmseg.datasets import build_dataloader
from mmseg.core.evaluation import mean_iou
from crfseg import CRF

In [5]:
cityspaces_path = repo_path.parent / "data" / "cityscapes"
device = "cuda"
batch_size = 2

seg_model = init_segmentor(config_file, checkpoint_file, device=device)
crf = CRF(n_spatial_dims=2, returns="log-proba").to(device)

cfg = seg_model.cfg
train_dataset = CityscapesDataset(
    data_root=cityspaces_path, 
    pipeline=cfg.data.train.pipeline, 
    img_dir=cfg.data.train.img_dir, 
    ann_dir=cfg.data.train.ann_dir, 
    test_mode=False
)

val_dataset = CityscapesDataset(
    data_root=cityspaces_path, 
    pipeline=cfg.data.val.pipeline, 
    img_dir=cfg.data.val.img_dir, 
    ann_dir=cfg.data.val.ann_dir, 
    test_mode=False
)

train_loader = build_dataloader(
    train_dataset, samples_per_gpu=batch_size, workers_per_gpu=0, dataloader_type="DataLoader")

val_loader = build_dataloader(
    val_dataset, samples_per_gpu=batch_size, workers_per_gpu=0, dataloader_type="DataLoader")

# Freeze Seg Network
for param in seg_model.parameters():
    param.requires_grad = False

optimizer = torch.optim.Adam(crf.parameters(), lr=1e-3)

2021-01-18 13:29:48,577 - mmseg - INFO - Loaded 2975 images
2021-01-18 13:29:48,585 - mmseg - INFO - Loaded 500 images


In [6]:
def get_evaluations(log_proba, seg, n_classes, ignore_idx):
    res = log_proba.argmax(1)
    miou, cate_acc, cate_iou = mean_iou(
        results=res,
        gt_seg_maps=seg, 
        num_classes=n_classes, 
        ignore_index=ignore_idx, 
        nan_to_num=0.0
    )
    # record
    return miou, cate_acc, cate_iou

def train_crf(train_loader, seg_model, crf, optimizer):
    batch_size = train_loader.batch_size
    train_dst = train_loader.dataset
    total_loss = 0
    total_miou = 0
    total_cate_acc = np.zeros(len(train_dst.CLASSES))
    total_cate_iou = np.zeros(len(train_dst.CLASSES))
    n_classes = len(train_dst.CLASSES)
    ignore_idx = train_dst.ignore_index
    n = 0
    
    pbar = tqdm(desc="[Train]", total=len(train_loader)*batch_size)
    seg_model.eval()
    crf.train()
    
    for data in train_loader:
        img = data["img"].data[0].to(device)
        img_meta = data["img_metas"].data[0]
        seg = data["gt_semantic_seg"].data[0].squeeze(1).to(device)

        optimizer.zero_grad()
        x = seg_model.inference(img, img_meta, rescale=False)
        log_proba = crf(x, display_tqdm=False)  # (B, K, H, W)
        loss = F.nll_loss(log_proba, seg, ignore_index=ignore_idx, reduction="mean")
        loss.backward()
        optimizer.step()

        # evaluations
        miou, cate_acc, cate_iou = get_evaluations(
            log_proba.cpu(), seg.cpu(), n_classes, ignore_idx
        )

        # record
        total_loss += loss.item()
        total_miou += miou
        total_cate_acc += cate_acc.round(4)
        total_cate_iou += cate_iou.round(4)
        
        n += 1

        pbar.update(batch_size)
        pbar.set_description(f"[Train] Loss {loss.item():.4f} | Mean IoU {miou:.4f}")
    pbar.close()
    
    return crf, total_loss/n, total_miou/n, total_cate_acc/n, total_cate_iou/n

def val_crf(val_loader, seg_model, crf):
    batch_size = val_loader.batch_size
    val_dst = val_loader.dataset
    total_loss = 0
    total_miou = 0
    total_cate_acc = np.zeros(len(val_dst.CLASSES))
    total_cate_iou = np.zeros(len(val_dst.CLASSES))
    n_classes = len(val_dst.CLASSES)
    ignore_idx = val_dst.ignore_index
    n = 0
    
    pbar = tqdm(desc="[Valid]", total=len(val_loader)*batch_size)
    seg_model.eval()
    crf.eval()
    with torch.no_grad():
        for data in val_loader:
            img = data["img"][0].data.to(device)
            img_meta = data["img_metas"][0].data[0]
            seg = data["gt_semantic_seg"][0].data.squeeze(1).long().to(device)

            x = seg_model.inference(img, img_meta, rescale=False)
            log_proba = crf(x, display_tqdm=False)
            loss = F.nll_loss(log_proba, seg, ignore_index=ignore_idx, reduction="mean")
            # evaluations
            miou, cate_acc, cate_iou = get_evaluations(
                log_proba.cpu(), seg.cpu(), n_classes, ignore_idx
            )
            # record
            total_loss += loss.item()
            total_miou += miou
            total_cate_acc += cate_acc.round(4)
            total_cate_iou += cate_iou.round(4)

            n += 1

            pbar.update(batch_size)
            pbar.set_description(f"[Valid] Loss {loss.item():.4f} | Mean IoU {miou:.4f}")
    pbar.close()
    
    return crf, total_loss/n, total_miou/n, total_cate_acc/n, total_cate_iou/n

In [7]:
n_step = 5
best_miou = 0.0
best_loss = 999999
crf_checkpoint_file = f"crf_{backbone_config}.pth"
sv_eval_path = Path("./eval_result.txt")
eval_file = sv_eval_path.open("w", encoding="utf-8")
eval_file.write("\t".join(train_dataset.CLASSES))

for step in range(n_step):
    crf, train_loss, train_miou, train_cate_acc, train_cate_iou = train_crf(
        train_loader, seg_model, crf, optimizer)
    torch.cuda.empty_cache()
    crf, val_loss, val_miou, val_cate_acc, val_cate_iou = val_crf(
        val_loader, seg_model, crf)
    print(f"[{step+1}/{n_step}] Train Loss: {train_loss:.4f} | Train Mean IoU: {train_miou:.4f}")
    print(f"[{step+1}/{n_step}] Val Loss: {val_loss:.4f} | Val Mean IoU: {val_miou:.4f}")
    # Save 
    if best_loss > val_loss:
        best_loss = val_loss
        best_miou = val_miou
        best_cate_acc = val_cate_acc
        best_cate_iou = val_cate_iou
        torch.save(crf.state_dict(), crf_checkpoint_file)
        print("[INFO] Saved")
        
eval_file.write("\t".join([f"{acc:.4f}" for acc in best_cate_acc]))
eval_file.write("\t".join([f"{iou:.4f}" for iou in best_cate_iou]))
eval_file.close()

[Train]:   0%|          | 0/2976 [00:00<?, ?it/s]/home/simonjisu/miniconda3/lib/python3.7/site-packages/crfseg/model.py:91: UserWarning: Mixed memory format inputs detected while calling the operator. The operator will output contiguous tensor even if some of the inputs are in channels_last format. (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629403081/work/aten/src/ATen/native/TensorIterator.cpp:918.)
  x = negative_unary - x
/home/simonjisu/miniconda3/lib/python3.7/site-packages/mmseg/core/evaluation/metrics.py:162: RuntimeWarning: invalid value encountered in true_divide
  acc = total_area_intersect / total_area_label
/home/simonjisu/miniconda3/lib/python3.7/site-packages/mmseg/core/evaluation/metrics.py:166: RuntimeWarning: invalid value encountered in true_divide
  iou = total_area_intersect / total_area_union
[Train] Loss 2.2906 | Mean IoU 0.4828: 100%|██████████| 2976/2976 [23:16<00:00,  2.13it/s]
[Valid] Loss 0.4199 | Mean IoU 0.9671: 100%|██████████| 500/500 [09:

[1/10] Loss: 1.3928 | Mean IoU: 0.7379
[1/10] Loss: 0.4485 | Mean IoU: 0.9623
[INFO] Saved


[Train] Loss 0.6419 | Mean IoU 0.9261: 100%|██████████| 2976/2976 [23:19<00:00,  2.13it/s]
[Valid] Loss 0.3212 | Mean IoU 0.9671: 100%|██████████| 500/500 [09:08<00:00,  1.10s/it]
[Train]:   0%|          | 0/2976 [00:00<?, ?it/s]

[2/10] Loss: 1.3277 | Mean IoU: 0.7355
[2/10] Loss: 0.3508 | Mean IoU: 0.9623


[Train] Loss 1.7770 | Mean IoU 0.6095: 100%|██████████| 2976/2976 [23:17<00:00,  2.13it/s]
[Valid] Loss 0.3996 | Mean IoU 0.9671: 100%|██████████| 500/500 [09:09<00:00,  1.10s/it]
[Train]:   0%|          | 0/2976 [00:00<?, ?it/s]

...

[Train] Loss 0.3488 | Mean IoU 0.9626: 100%|██████████| 2976/2976 [23:17<00:00,  2.13it/s]
[Valid] Loss 0.3359 | Mean IoU 0.9671: 100%|██████████| 500/500 [09:09<00:00,  1.10s/it]
[Train]:   0%|          | 0/2976 [00:00<?, ?it/s]

[9/10] Loss: 1.3242 | Mean IoU: 0.7359
[9/10] Loss: 0.3654 | Mean IoU: 0.9623


[Train] Loss 1.3293 | Mean IoU 0.7208: 100%|██████████| 2976/2976 [23:17<00:00,  2.13it/s]
[Valid] Loss 0.3012 | Mean IoU 0.9671: 100%|██████████| 500/500 [09:07<00:00,  1.10s/it]

[10/10] Loss: 1.2954 | Mean IoU: 0.7429
[10/10] Loss: 0.3309 | Mean IoU: 0.9623


AttributeError: '_io.TextIOWrapper' object has no attribute 'closse'